In [1]:
import os
import numpy as np
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator

m = MPRester('Yct0KDbJbqMLWluZEovkwrLXh2VRHXbc')

/home/gterejan/.local/lib/python3.10/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [2]:
config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
checkpoint_path = "dimenetpp_all.pt"

# Define the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)

ERROR:root:Invalid setup for SCN. Either the e3nn library or Jd.pt is missing.


amp: false
cmd:
  checkpoint_dir: /host_workspace/checkpoints/2023-07-07-10-17-36
  commit: null
  identifier: ''
  logs_dir: /host_workspace/logs/tensorboard/2023-07-07-10-17-36
  print_every: 100
  results_dir: /host_workspace/results/2023-07-07-10-17-36
  seed: null
  timestamp_id: 2023-07-07-10-17-36
dataset: null
gpus: 0
logger: tensorboard
model: dimenetplusplus
model_attributes:
  cutoff: 6.0
  hidden_channels: 256
  num_after_skip: 2
  num_before_skip: 1
  num_blocks: 3
  num_output_layers: 3
  num_radial: 6
  num_spherical: 7
  otf_graph: true
  out_emb_channels: 192
  regress_forces: false
  use_pbc: true
noddp: false
optim:
  batch_size: 4
  eval_batch_size: 4
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 115082
  - 230164
  - 345246
  max_epochs: 20
  num_workers: 4
  warmup_factor: 0.2
  warmup_steps: 57541
slurm: {}
task:
  dataset: single_point_lmdb
  description: Relaxed state energy prediction from initial structure.
  labels:
  - relaxed energy
  metric: m

In [20]:
fingerprint_ads = ['H', 'O', 'C', 'CO', 'OH',    'N2', 'CO2', 'NH3', 'CH4', 'H2S']

bulk_symbol = "Pt"
ads_symbol = fingerprint_ads[1]
miller_index = (1, 1, 1)

In [21]:
# get material ids for a given bulk symbol
material_ids = m.get_materials_ids(bulk_symbol)

Retrieving MaterialsDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
print(material_ids)

[MPID(mp-126), MPID(mp-2646979), MPID(mp-2647022)]


In [32]:
material_id = material_ids[0]

# get the slabs for a given material_id
structure = m.get_structure_by_material_id(material_id)

# Create the SlabGenerator
slabgen = SlabGenerator(structure, miller_index, 10, 10)

# Get all slabs
slabs = slabgen.get_slabs()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
len(slabs)

1

In [34]:
slab = slabs[0]

# Construct a sample structure
# adslab = surface(bulk_symbol, miller_index, layers=3)

adslab = AseAtomsAdaptor().get_atoms(slab)

adsorbate = molecule(ads_symbol)
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)
cons= FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)

# Set up the calculator
adslab.calc = calc

# get adsorption energy
adslab.get_potential_energy()

0.509479284286499

In [14]:
m.get_structure_by_material_id(material_ids[0])

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Structure Summary
Lattice
    abc : 4.14407046 4.144070993962987 5.47557816
 angles : 90.0 90.0 119.99999573768088
 volume : 81.4357219320045
      A : 4.14407046 0.0 0.0
      B : -2.07203523 3.58887091 0.0
      C : 0.0 -0.0 5.47557816
    pbc : True True True
PeriodicSite: Sn (2.0720, 1.1963, 4.1067) [0.6667, 0.3333, 0.7500]
PeriodicSite: Sn (-0.0000, 2.3926, 1.3689) [0.3333, 0.6667, 0.2500]
PeriodicSite: Pt (0.0000, 0.0000, 2.7378) [-0.0000, 0.0000, 0.5000]
PeriodicSite: Pt (0.0000, 0.0000, 0.0000) [-0.0000, 0.0000, 0.0000]

In [16]:
slabs[0]

Structure Summary
Lattice
    abc : 4.144070993962987 5.47557816 24.86442276
 angles : 90.0 119.99999573768088 90.0
 volume : 488.61433159202704
      A : 3.58887091 0.0 -2.0720352299999996
      B : 8.805401259170722e-16 5.47557816 3.3528246335625773e-16
      C : 0.0 0.0 24.86442276
    pbc : True True True
PeriodicSite: Sn (1.1963, 4.1067, 1.3814) [0.3333, 0.7500, 0.0833]
PeriodicSite: Sn (2.3926, 1.3689, -0.6907) [0.6667, 0.2500, 0.0278]
PeriodicSite: Pt (0.0000, 2.7378, 3.4534) [0.0000, 0.5000, 0.1389]
PeriodicSite: Pt (0.0000, 0.0000, 3.4534) [0.0000, 0.0000, 0.1389]
PeriodicSite: Sn (1.1963, 4.1067, 5.5254) [0.3333, 0.7500, 0.2500]
PeriodicSite: Sn (2.3926, 1.3689, 3.4534) [0.6667, 0.2500, 0.1944]
PeriodicSite: Pt (0.0000, 2.7378, 7.5975) [0.0000, 0.5000, 0.3056]
PeriodicSite: Pt (0.0000, 0.0000, 7.5975) [0.0000, 0.0000, 0.3056]
PeriodicSite: Sn (1.1963, 4.1067, 9.6695) [0.3333, 0.7500, 0.4167]
PeriodicSite: Sn (2.3926, 1.3689, 7.5975) [0.6667, 0.2500, 0.3611]
PeriodicSite: Pt (

In [17]:
slabs[1]

Structure Summary
Lattice
    abc : 4.144070993962987 5.47557816 24.86442276
 angles : 90.0 119.99999573768088 90.0
 volume : 488.61433159202704
      A : 3.58887091 0.0 -2.0720352299999996
      B : 8.805401259170722e-16 5.47557816 3.3528246335625773e-16
      C : 0.0 0.0 24.86442276
    pbc : True True True
PeriodicSite: Sn (1.1963, 4.1067, 0.0000) [0.3333, 0.7500, 0.0278]
PeriodicSite: Sn (2.3926, 1.3689, 2.0720) [0.6667, 0.2500, 0.1389]
PeriodicSite: Pt (0.0000, 2.7378, 2.0720) [0.0000, 0.5000, 0.0833]
PeriodicSite: Pt (0.0000, 0.0000, 2.0720) [0.0000, 0.0000, 0.0833]
PeriodicSite: Sn (1.1963, 4.1067, 4.1441) [0.3333, 0.7500, 0.1944]
PeriodicSite: Sn (2.3926, 1.3689, 6.2161) [0.6667, 0.2500, 0.3056]
PeriodicSite: Pt (0.0000, 2.7378, 6.2161) [0.0000, 0.5000, 0.2500]
PeriodicSite: Pt (0.0000, 0.0000, 6.2161) [0.0000, 0.0000, 0.2500]
PeriodicSite: Sn (1.1963, 4.1067, 8.2881) [0.3333, 0.7500, 0.3611]
PeriodicSite: Sn (2.3926, 1.3689, 10.3602) [0.6667, 0.2500, 0.4722]
PeriodicSite: Pt (